# CNN for sentence classification
- [Reference code](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb) by bentrevett
- Dataset [download](https://github.com/yoonkim/CNN_sentence)

In [1]:
import os
import pandas as pd
import numpy as np
import random
import torch

In [2]:
def set_seed(seed= 1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

## Data preparation

### Data load

In [3]:
# 데이터 준비 함수 (None -> df)
def load_data():
    neg_path = r"D:\SIMON\대학원\9_스터디\2021_pytorch_study\1월_CNN\1_reference_code\rt-polarity.neg.txt"
    neg_df = pd.read_table(neg_path, header=None, names=['X'], encoding='latin') #'ISO-8859-1' 의 alias
    neg_df['y'] = [0] * len(neg_df)

    pos_path = r"D:\SIMON\대학원\9_스터디\2021_pytorch_study\1월_CNN\1_reference_code\rt-polarity.pos.txt"
    pos_df = pd.read_table(pos_path, header=None, names=['X'], encoding='latin') #'ISO-8859-1' 의 alias
    pos_df['y'] = [1] * len(pos_df)

    data = pd.concat([neg_df, pos_df], axis=0)
    data.reset_index(inplace=True)
    print("# of Loaded Data: {}".format(data.shape[0]))
    
    return data

In [4]:
data = load_data()

# of Loaded Data: 10662


### Preprocessing

In [5]:
import re
from nltk.corpus import stopwords  

In [6]:
#텍스트 전처리 함수 (str -> lst)
def text_preprocessor(sent):
    stop = stopwords.words('english')

    # text preprocessing
    sent_str = sent.lower()
    sent_str = re.sub('[^a-z]', ' ', sent_str)  # Remove non-alphabetic strings
    sent_str = re.sub('  ', ' ', sent_str).strip()  # Remove double white spaces
    sent_lst = [word for word in sent_str.split() if word not in stop] # Remove stopwords
    return sent_lst

In [7]:
data['X_lst'] = data['X'].map(text_preprocessor)

### Data split

In [9]:
# 데이터 분리 함수 (df -> [df,df,df])
def data_split(df):
    train_df = df.sample(frac=0.8, random_state=123)
    test_df = df.drop(train_df.index)

    val_df = train_df.sample(frac=0.2, random_state=123)
    train_df = train_df.drop(val_df.index)
    
    print("Train_df shape:", train_df.shape)
    print("Val_df shape:", val_df.shape)
    print("Test_df shape:", test_df.shape)

    return train_df, val_df, test_df

In [10]:
train_df, val_df, test_df = data_split(data)

Train_df shape: (6824, 4)
Val_df shape: (1706, 4)
Test_df shape: (2132, 4)


### Build Vocab

In [11]:
# Vocab 만들어주는 함수 ([df,df] -> [dict, dict])
def buildVocab(train_df, val_df):
    '''train과 valid 데이터만 사용 주의'''
    trainWords_lst = [w for w_lst in train_df['X_lst'] for w in w_lst]
    ValWords_lst = [w for w_lst in val_df['X_lst'] for w in w_lst]
    totalWords_lst = trainWords_lst + ValWords_lst

    words_lst = ['<pad>', '<unk>'] + sorted(list(set(totalWords_lst)))

    itos = {idx : word for idx, word in enumerate(words_lst)}
    stoi = {word : idx for idx, word in enumerate(words_lst)}

    print('length of word_set:', len(words_lst))
    return itos, stoi

In [12]:
itos, stoi = buildVocab(train_df, val_df)

length of word_set: 16278


### String to Index (Numericalization)

In [13]:
# 문장을 숫자로 표현해주는 함수 (lst -> lst)
def token_to_idx(token_lst, stoi):
    idx_lst = []
    for w in token_lst:
        if w in stoi:
            idx = stoi[w]
        else:
            idx = stoi['<unk>']
        idx_lst.append(idx)
    return idx_lst

In [14]:
train_df['X_idx'] = train_df['X_lst'].map(lambda x: token_to_idx(x, stoi))
val_df['X_idx'] = val_df['X_lst'].map(lambda x: token_to_idx(x, stoi))
test_df['X_idx'] = test_df['X_lst'].map(lambda x: token_to_idx(x, stoi))

### Padding

In [15]:
# zero padding을 통해서 시퀀스의 길이를 맞춰주는 함수 ([lst, int, dict] -> lst)
def zero_padding(idx_lst, max_len, stoi):
    pad_idx = stoi['<pad>']
    unk_idx = stoi['<unk>']
    
    idx_lst = idx_lst[:max_len]
    
    if len(idx_lst) == max_len:
        return idx_lst
    else:
        padding_len = max_len - len(idx_lst)
        padding_list = [pad_idx] * padding_len
        idx_lst = idx_lst + padding_list
        return idx_lst

In [16]:
train_df['X_pad'] = train_df['X_idx'].map(lambda x: zero_padding(x, 10, stoi))
val_df['X_pad'] = val_df['X_idx'].map(lambda x: zero_padding(x, 10, stoi))
test_df['X_pad'] = test_df['X_idx'].map(lambda x: zero_padding(x, 10, stoi))

### torch DataLoader

In [100]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x # np.array 말고 list로도 가능?
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        input_x = torch.LongTensor(self.x[idx])
        target_y = torch.FloatTensor([self.y[idx]]) # 왜 FloatTensor?,torch.FloatTensor([]) 이렇게 넣어줘야하는듯.
        return input_x, target_y

In [101]:
# 기존 idx를 제거하기 위해서 list() 사용
trainset = CustomDataset(list(train_df['X_pad']), list(train_df['y']))
validset = CustomDataset(list(val_df['X_pad']), list(val_df['y']))
testset = CustomDataset(list(test_df['X_pad']), list(test_df['y']))

In [102]:
from torch.utils.data import DataLoader

batch_size = 64
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=False)
validloader = DataLoader(validset, batch_size=batch_size, shuffle=True, drop_last=False)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True, drop_last=False)

## Build CNN model
- In pytorch, CNN wants the batch dimension first!
- in_channels: # of channels / out_channels: # of filters / kernel_size: filter size

In [84]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [85]:
import torch.nn as nn
import torch.nn.functional as F

In [167]:
class CNN(nn.Module):
    def __init__(self, pretrained_embeddings, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze= True, padding_idx= pad_idx)
        
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
        self.conv_2 = nn.Conv2d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim) # Because of simply "CONCATENATE"!
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        # text = [batch_size, sent_len]
        print("text:", text.shape)
        print("text:", text.dtype)
        embedded = self.embedding(text) 
        print("embedded after embedding:", embedded.shape)
        print("embedded after embedding:", embedded.dtype)
        # embedded = [batch_size, sent_len, embed_dim]
        embedded = embedded.unsqueeze(1) # Insert 1 dimension to represent # of channels like images
        print("embedded after unsqueeze:", embedded.shape)
        print("embedded after unsqueeze:", embedded.dtype)
        
        test = self.conv_0(embedded)
        
        # embedded = [batch_size, 1, sent_len, embed_dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
        print("conved_0:", conved_0.shape)
        print("conved_0:", conved_0.dtype)
        
        # conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        # pooled_n = [batch_size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
        
        # cat = [batch_size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [156]:
class CNN(nn.Module):
    def __init__(self, pretrained_embeddings, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze= True, padding_idx= pad_idx)
        
        # Note List comprehension
        self.convs = nn.ModuleList([nn.Conv2d(in_channels= 1,  # 1 channel for TEXT
                                              out_channels= n_filters,
                                              kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim) # Because of simply "CONCATENATE"!
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        # text = [batch_size, sent_len]
        print("text:", text.shape)
        print("text:", text.dtype)
        embedded = self.embedding(text) 
        print("embedded after embedding:", embedded.shape)
        print("embedded after embedding:", embedded.dtype)
        # embedded = [batch_size, sent_len, embed_dim]
        embedded = embedded.unsqueeze(1) # Insert 1 dimension to represent # of channels like images
        print("embedded after unsqueeze:", embedded.shape)
        print("embedded after unsqueeze:", embedded.dtype)
        
        # embedded = [batch_size, 1, sent_len, embed_dim]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]  # WHY does Squeeze need here?
        print("conved:", conved.shape)
        print("conved:", conved.dtype)
        
        # conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        # pooled_n = [batch_size, n_filters]
        cat = self.dropout(torch.cat(pooled, dim=1))
        
        # cat = [batch_size, n_filters * len(filter_sizes)]
        return self.fc(cat)

In [157]:
t = torch.FloatTensor([1,2,3])

In [158]:
t.type()

'torch.FloatTensor'

### Load pre-trained embedding
- Load the pretrain Word2Vec model from Google [(Download HERE)](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit) 
- It might take time since it  contains 300-dimensional vectors for 3 million words and phrases

In [107]:
from gensim.models.keyedvectors import KeyedVectors

In [108]:
# Load pretrained_embeddings
path = r"C:\Users\Simon\ongoing_projects\SSRC_collaboration\source\GoogleNews-vectors-negative300.bin"
word_vectors = KeyedVectors.load_word2vec_format(path, binary=True)

# Customize pretrained_embeddings 
pretrained_embeddings = np.zeros((len(stoi), 300)) # <pad>, <unk>은 여기서는 제외시켜야 하나??

# unk_lst = []
for i, w in enumerate(list(stoi.keys())):
    try:
        tmp_vec = word_vectors[w]
        pretrained_embeddings[i] = tmp_vec
    except KeyError:
        pass
#         unk_lst.append(i)

In [109]:
# unk_lst.remove(0) # <pad>
# unk_lst.remove(1) # <unk>
# pretrained_embeddings = np.delete(pretrained_embeddings, unk_lst, axis=0) # <unk>이 있기때문에 없는 단어들 제거????

In [127]:
# config
# INPUT_DIM = len(stoi)
PRETRAINED_EMB = torch.from_numpy(pretrained_embeddings) # torch.tensor로 전환 주의
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = stoi['<pad>']

In [168]:
model = CNN(PRETRAINED_EMB, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [169]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 360,601 trainable parameters


## Train the model

In [170]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [171]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [172]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for x, y in iterator:
        x = x.to('cuda')
        y = y.to('cuda')
        
        optimizer.zero_grad()
        
        predictions = model(x).squeeze(1)
        loss = criterion(predictions, y)
        acc = binary_accuracy(predictions, y)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [173]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for x, y in iterator:
            x = x.to('cuda')
            y = y.to('cuda')
            
            predictions = model(x).squeeze(1)
            loss = criterion(predictions, y)
            acc = binary_accuracy(predictions, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [174]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [175]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, trainloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, validloader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

text: torch.Size([64, 10])
text: torch.int64
embedded after embedding: torch.Size([64, 10, 300])
embedded after embedding: torch.float64
embedded after unsqueeze: torch.Size([64, 1, 10, 300])
embedded after unsqueeze: torch.float64


RuntimeError: Input type (torch.cuda.DoubleTensor) and weight type (torch.cuda.FloatTensor) should be the same